In [1]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import numpy as np


In [2]:
# Extracting full post text and group number from scraped paged and adding to df

In [3]:
txt = open("post4.txt","r") 
long_soup = txt.read()

In [4]:
# Extract post number

def extract_post_num (txt):
    if txt != 'blank_data':
        epn = re.findall (r'activity-item-body-text-(.*)"', str(txt))
        return epn[0]
    else:
        return np.nan        

extract_post_num(long_soup)

'104230'

In [5]:
x = re.split (extract_post_num(long_soup) , str(long_soup))
x[15]

'">\n<p>No legal support from military after I caught my husband cheating. Legal told me not to say anything because he is the victim. SERIOUSLY<br/>\nhe cleared bank accounts, changed all passwords, closed my 2 year old granddaughter college fun and is out having an affair and he is the victim. Left me an account with 200 dollars, no idea what to do now.</p>\n</span>\n</div>\n<span class="comment-postbutton" id="comment-postbutton-'

In [6]:
# Extract OP...

def extract_op (txt):
    if txt != 'blank_data':
        #op = re.split (extract_post_num(txt) , str(txt))
        op = re.findall (r'id="activity-item-body-text-(.+?)</p>', txt, flags= re.DOTALL)
        op = re.sub(r'[^a-zA-Z ]+', '', str(op))
        return op
    else:
        return np.nan
    
extract_op(long_soup)

'npNo legal support from military after I caught my husband cheating Legal told me not to say anything because he is the victim SERIOUSLYbrnhe cleared bank accounts changed all passwords closed my  year old granddaughter college fun and is out having an affair and he is the victim Left me an account with  dollars no idea what to do now'

In [7]:
re.findall (r'<span class="in-group"> in (.+?)<', long_soup, flags= re.DOTALL)

['<a href="https://military-family.supportgroups.com">Military Family']

In [8]:
def extract_groups(txt):
    if txt != 'blank_data':
        eg = re.findall (r'<span class="in-group"> in (.+?)</span>', txt, flags= re.DOTALL)
        eg = re.findall (r'[A-Z].*?(?=<)', str(eg), flags= re.DOTALL)
        return eg
    else:
        return np.nan
    
extract_groups (long_soup)

['Military Family']

In [9]:
# Main function to open and perform other functions on each text file.  
#Result will be df with cols post_number, original_post, responses

def whole_post (name = 'post', file_type = '.txt', start_num = 0, end_num = 35157):
    post_number = []
    groups = []
    pst = []
    for i in range(start_num, end_num+1):
        fl = name + str(i) + file_type
        fl = open(fl,"r") 
        post = fl.read()
        if post != 'blank_data':
            epn = extract_post_num (post)
            eg = extract_groups(post)
            ep = extract_op(post)
            post_number.append(epn)
            groups.append(eg)
            pst.append(ep)
            
        else:
            post_number.append(np.nan)
            groups.append(np.nan)
            pst.append(np.nan)

        fl.close()
    df = pd.DataFrame(
                     {'post_number': post_number,
                      'long_post' : pst,
                      'groups': groups
                      })
    return df


df = whole_post()



FileNotFoundError: [Errno 2] No such file or directory: 'post5.txt'

In [ ]:
# pickle.dump(df, open('Clayton_SG_FullPost_df.pkl', 'bw'))

In [ ]:
df.head()

In [ ]:
df.long_post[15642].split()[0]

In [ ]:
# Create col of first pords to identify bad post

def bad_post(col = df['long_post']):
    fw = []
    b = col
    for i in range (0, len(b)):
        wrd = str(b[i]).split()[0]
        fw.append(wrd)
    return fw

df['first_word'] = bad_post()
print(df.shape)
df.head()



In [ ]:
#  drop bad post and col for filtering

df = df[df['first_word'] != 'nspanndivnspan']
df = df.drop(columns = ['first_word'])
print(df.shape)
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()
df.isna().sum()

In [ ]:
df = df.reset_index(drop=True)
df.shape

In [ ]:
def np_gone(col = df['long_post']):
    nnp = []
    for i in range (0, len(col)):
        if col[i][0:2] == 'np':
            nnp.append(col[i][2:])
        else:
            pass
    return nnp

df['long_post'] = np_gone()
df.head()